## URL extraction

In [27]:
import pandas as pd
import requests
import numpy as np 
from bs4 import BeautifulSoup
import re

data = pd.read_csv('sample_port.csv')

names = data['Scheme Name']



def check_url_for_404(url):
    try:
        response = requests.get(url)
        if response.status_code == 404:
            return True
        else:
            return False
    except requests.exceptions.RequestException as e:
        print("An error occurred:", e)
        return False

# Example usage

scheme_url = []
NAV_list = []

for name in names:
    name1 = name.lower().replace(' ','-').replace('---','-').replace('regular','direct')
    
    
    name2 = name1.replace('plan','growth')

    # remove the string after growth 
    name3 = name2.split('growth', 1)[0].replace('-direct-','-direct-growth').replace('--','-').replace('-idcw-reinvestment','')

    if 'fund-of-fund' in name3:
        name3 = name3.replace('fund-of-fund','fof')

    if name3.endswith('fund-'):
        name3 = name3.replace('fund-','fund-direct-growth')
    
    if name3.endswith('fund'):
        name3 = name3.replace('fund','fund-direct-growth')
    
    if name3.endswith('discovery-'):
        name3 = name3.replace('discovery-','discovery-fund-direct-growth')
    
    if name3=='icici-prudential-large-&-mid-cap-fund-direct-growth':
        name3 = 'icici-prudential-top-100-fund-direct-growth'
    
    if name3=='icici-prudential-bluechip-fund-direct-growth':
        name3 = 'icici-prudential-focused-bluechip-equity-fund-direct-growth' 
    
    if name3=='sbi-large-&-midcap-fund-direct-growth':
        name3 = 'sbi-magnum-multiplier-fund-direct-growth'

    if name3=='kotak-emerging-equity-fund':
        name3 = 'kotak-emerging-equity-scheme-direct-growth'

    if name3=='quant-small-cap-fund-direct-growth':
        name3 = 'quant-small-cap-fund-direct-plan-growth'

    if name3=='dsp-flexi-cap-fund-direct-growth':
        name3 = 'dsp-blackrock-equity-fund-direct-growth'

    if name3=='kotak-emerging-equity-fund-direct-growth':
        name3 = 'kotak-emerging-equity-scheme-direct-growth'

    if name3=='kotak-small-cap-fund-direct-growth':
        name3= 'kotak-midcap-fund-direct-growth'

    if name3=='templeton-india-equity-income-fund-direct-growth':
        name3 = 'templeton-india-equity-i-come-fund-direct-growth'

    if name3=='dsp-banking-&-financial-services-fund-direct-growth':
        name3 = 'dsp-banking-financial-services-fund-direct-growth'

    url = f"https://groww.in/mutual-funds/{name3}"
    print(url)
    is_404 = check_url_for_404(url)
    if is_404:
        print(name3)

        print(f"The URL {url} returns a 404 status code.")
    
    urldat = requests.get(url)

    soup = BeautifulSoup(urldat.text, 'html.parser')



    td_tag = soup.find_all('td',class_="fd12Cell contentPrimary bodyLargeHeavy")[0]

    # Extract the text content from the <td> tag
    value = td_tag.get_text(strip=True)

    NAV= re.search(r'₹([\d.]+)', value).group(1)

    NAV_list.append(NAV)
    scheme_url.append(url)

    #.split('-')[0].lower().replace(' ','-')[:-1])

data['scheme_url'] = np.array(scheme_url)
data['NAV'] = np.array(NAV_list)

data.to_csv('sample_port.csv',index=False)


https://groww.in/mutual-funds/quant-momentum-fund-direct-growth
https://groww.in/mutual-funds/hdfc-transportation-and-logistics-fund-direct-growth
https://groww.in/mutual-funds/quant-momentum-fund-direct-growth
https://groww.in/mutual-funds/axis-india-manufacturing-fund-direct-growth
https://groww.in/mutual-funds/icici-prudential-top-100-fund-direct-growth
https://groww.in/mutual-funds/quant-commodities-fund-direct-growth
https://groww.in/mutual-funds/kotak-healthcare-fund-direct-growth
https://groww.in/mutual-funds/hdfc-mid-cap-opportunities-fund-direct-growth
https://groww.in/mutual-funds/hdfc-technology-fund-direct-growth
https://groww.in/mutual-funds/icici-prudential-focused-bluechip-equity-fund-direct-growth
https://groww.in/mutual-funds/franklin-india-flexi-cap-fund-direct-growth
https://groww.in/mutual-funds/icici-prudential-multicap-fund-direct-growth
https://groww.in/mutual-funds/nippon-india-multi-cap-fund-direct-growth
https://groww.in/mutual-funds/groww-nifty-total-market-i

## Data 

In [28]:
import pandas as pd 

data = pd.read_csv('sample_port.csv')

columns_to_keep = ['Scheme Name', 'ISIN', 'Scheme Category Name', 'Units', 'NAV','scheme_url']

data = data[columns_to_keep]

data.to_csv('sample_port.csv',index=False)
data.head()

,Scheme Name,ISIN,Scheme Category Name,Units,NAV,scheme_url
0,quant Momentum Fund - Regular Plan IDCW - Rein...,INF966L01CY6,Equity - Diversified,3000.000,13.73,https://groww.in/mutual-funds/quant-momentum-f...
1,HDFC Transportation and Logistics Fund Regular...,INF179KC1GX4,Equity - Sector - Auto,1500.000,13.49,https://groww.in/mutual-funds/hdfc-transportat...
2,quant Momentum Fund - Regular Plan Growth - Gr...,INF966L01CW0,Equity - Diversified,7075.965,13.73,https://groww.in/mutual-funds/quant-momentum-f...
3,Axis India Manufacturing Fund Regular Growth,INF846K013S0,Equity - Diversified,3000.000,11.75,https://groww.in/mutual-funds/axis-india-manuf...
4,ICICI Prudential Large & Mid Cap Fund - Growth,INF109K01431,Equity - Diversified,220.345,924.99,https://groww.in/mutual-funds/icici-prudential...


## Data get the holding

In [73]:
import requests
from bs4 import BeautifulSoup
import json

data = pd.read_csv('sample_port.csv')


url = requests.get(data.scheme_url[3])

soup = BeautifulSoup(url.text, 'html.parser')


# Find the script tag with the specific ID
script_tag = soup.find('script', id='__NEXT_DATA__')

# Extract the JSON data from the script tag content
json_data = json.loads(script_tag.contents[0])

# get holdings
holdings = json_data['props']['pageProps']['mf']['holdings']

# get pandas
df = pd.DataFrame(holdings)


In [45]:
data['Scheme Category Name']

0             Equity - Diversified
1           Equity - Sector - Auto
2             Equity - Diversified
3             Equity - Diversified
4             Equity - Diversified
5             Equity - Diversified
6         Equity - Sector - Pharma
7             Equity - Diversified
8     Equity - Sector - Technology
9             Equity - Diversified
10            Equity - Diversified
11            Equity - Diversified
12            Equity - Diversified
13                     Index Funds
14            Equity - Diversified
15            Equity - Diversified
16            Equity - Diversified
17            Equity - Diversified
18            Fund of Funds - Gold
Name: Scheme Category Name, dtype: object

In [74]:
import plotly.express as px
import altair as alt 

# Calculate counts of each category
category_counts = data['Scheme Category Name'].value_counts()

# Create a pie chart
fig = px.pie(values=category_counts.values, names=category_counts.index, title='Pie Chart of Categories')


fig.show()

In [76]:

# Calculate counts of each category
# category_counts = df['category'].value_counts().reset_index()

# Rename columns for clarity
category_counts = data['Scheme Category Name'].value_counts().reset_index()
# Rename columns for clarity
category_counts.columns = ['category', 'count']
alt.Chart(category_counts).mark_arc(innerRadius=50).encode(
    theta="count",
    color="category:N",
)

alt.Chart(...)